In [1]:
import random
import string
import secrets
import time
import concurrent.futures

from uuid import uuid4
from datetime import datetime

import requests
from tqdm import trange

In [2]:
def create_user(user_id: str | None = None):
    user_id = user_id or uuid4().hex
    username = "".join(random.choices(string.ascii_letters, k=10))
    name = "".join(random.choices(string.ascii_letters, k=10))
    res = requests.post("http://localhost:8888/users", json={
        "id": user_id,
        "username": username,
        "name": name,
        "password_hash": secrets.token_hex(16),
        "created": datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"),
    })
    assert res.status_code == 200

def create_post(user: str):
    post_id = uuid4().hex
    content = "".join(random.choices(string.ascii_letters, k=64))
    res = requests.post("http://localhost:8888/posts", json={
        "id": post_id,
        "user_id": user,
        "content": content,
        "created": datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f"),
    })
    assert res.status_code == 200

def create_user_and_posts(n_posts: int = 10):
    user_id = uuid4().hex
    create_user(user_id)

    for _ in range(n_posts):
        create_post(user_id)

def get_users():
    res = requests.get("http://localhost:8888/users")
    assert res.status_code == 200

In [6]:
create_user()

In [23]:
for _ in trange(1000):
    get_users()

100%|██████████| 1000/1000 [00:02<00:00, 357.36it/s]


In [24]:
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as pool:
    for _ in trange(1000):
        pool.submit(get_users)

100%|██████████| 1000/1000 [00:00<00:00, 1577.80it/s]


In [8]:
create_user()

In [26]:
for _ in trange(1000):
    create_user()

100%|██████████| 1000/1000 [00:04<00:00, 215.04it/s]


In [10]:
for _ in trange(1000):
    create_user()

100%|██████████| 1000/1000 [00:58<00:00, 17.17it/s]


In [18]:
for _ in trange(1000):
    create_user()

100%|██████████| 1000/1000 [00:58<00:00, 17.06it/s]


In [146]:
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as pool:
    for _ in trange(1000):
        pool.submit(create_user)

100%|██████████| 1000/1000 [00:00<00:00, 7568.60it/s]


In [4]:
create_post("b1a6851b83a34f3da9436d483d852e49")

In [19]:
for _ in trange(1000):
    create_post("5ab0e437eef04ba2967f9f8ccb4a266f")

100%|██████████| 1000/1000 [00:04<00:00, 233.95it/s]


In [27]:
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as pool:
    for _ in trange(1000):
        pool.submit(create_post, "88d9eb5e1bc74e0aafc2ebbfb6158ef7")

100%|██████████| 1000/1000 [00:00<00:00, 6785.18it/s]


In [8]:
for _ in trange(100):
    create_user_and_posts()

100%|██████████| 100/100 [00:13<00:00,  7.54it/s]


In [9]:
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as pool:
    for _ in trange(100):
        pool.submit(create_user_and_posts)

100%|██████████| 100/100 [00:00<00:00, 1310.13it/s]
